<a href="https://colab.research.google.com/github/DanYT2/machine_learning/blob/main/Naive_Bayes_vs_LR_vs_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from sklearn.svm import LinearSVC as SVM
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.linear_model import LogisticRegression as LR

from sklearn.metrics import accuracy_score

In [ ]:
dataset = load_dataset('rotten_tomatoes')
dataset

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [ ]:
Xtr = dataset['train']['text']
ytr = dataset['train']['label']

Xts = dataset['test']['text']
yts = dataset['test']['label']

Xval = dataset['validation']['text']
yval = dataset['validation']['label']

In [ ]:
texts = Xtr + Xts + Xval
print(len(texts))

10662


In [ ]:
tok = TfidfVectorizer(max_features=100)
tok.fit(texts)

train_seq = tok.transform(Xtr).toarray()
test_seq = tok.transform(Xts).toarray()
val_seq = tok.transform(Xval).toarray()

train_labels = np.array(ytr)
test_labels = np.array(yts)
val_labels = np.array(yval)


In [ ]:
# LR
lrclf = LR()
lrclf.fit(train_seq, train_labels)
lrpred = lrclf.predict(test_seq)
print("LR ACC: ", accuracy_score(test_labels, lrpred))

# SVM
svmclf = SVM()
svmclf.fit(train_seq, train_labels)
svmpred = svmclf.predict(test_seq)
print("SVM ACC: ", accuracy_score(test_labels, svmpred))

# NB
nbclf = MNB()
nbclf.fit(train_seq, train_labels)
nbpred = nbclf.predict(test_seq)
print("NB ACC: ", accuracy_score(test_labels, nbpred))

LR ACC:  0.6069418386491557
SVM ACC:  0.6078799249530957
NB ACC:  0.6116322701688556


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(128, activation = 'relu', input_shape = (train_seq.shape[1], )))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

hist = model.fit(
    train_seq,
    train_labels,
    validation_data = (val_seq, val_labels),
    epochs = 20,
    batch_size = 16
)


Epoch 1/20
534/534 [==============================] - 4s 3ms/step - loss: 0.6628 - accuracy: 0.6042 - val_loss: 0.6576 - val_accuracy: 0.6229
Epoch 2/20
534/534 [==============================] - 1s 2ms/step - loss: 0.6323 - accuracy: 0.6377 - val_loss: 0.6554 - val_accuracy: 0.6116
Epoch 3/20
534/534 [==============================] - 1s 2ms/step - loss: 0.6243 - accuracy: 0.6449 - val_loss: 0.6599 - val_accuracy: 0.6051
Epoch 4/20
534/534 [==============================] - 1s 2ms/step - loss: 0.6171 - accuracy: 0.6529 - val_loss: 0.6584 - val_accuracy: 0.6144
Epoch 5/20
534/534 [==============================] - 1s 2ms/step - loss: 0.6097 - accuracy: 0.6611 - val_loss: 0.6582 - val_accuracy: 0.6088
Epoch 6/20
534/534 [==============================] - 1s 2ms/step - loss: 0.6020 - accuracy: 0.6713 - val_loss: 0.6593 - val_accuracy: 0.6135
Epoch 7/20
534/534 [==============================] - 1s 2ms/step - loss: 0.5921 - accuracy: 0.6773 - val_loss: 0.6594 - val_accuracy: 0.6041
Epoch 

In [ ]:
loss, accuracy = model.evaluate(test_seq, test_labels)
print(loss)
print(accuracy)

34/34 [==============================] - 0s 2ms/step - loss: 0.6856 - accuracy: 0.6313
0.6855639815330505
0.6313320994377136
